#### Imports

In [ ]:
import numpy as np
import random 
import math
import time

#### Functions

- Distance Function

In [ ]:
def find_distance(x1,x2,y1,y2):
    return math.sqrt((x1-x2)**2+(y1-y2)**2)

- Cost Function

In [ ]:
def find_cost(supply_coords,demand_coords):
    cost = np.zeros((len(supply_coords),len(demand_coords)))
    for i in range(len(supply_coords)):
        for j in range(len(demand_coords)):
            cost[i,j] = find_distance(supply_coords[i][0],demand_coords[j][0],supply_coords[i][1],demand_coords[j][1])
    return cost

- Bubble Sort

In [ ]:
def bubble_sort(arrx,cost):
    parents = []
    for i in range(len(arrx)):
        parents.append(arrx[i][1])
    swapped = True
    while swapped:
        swapped = False
        for i in range(len(arrx) - 1):
            if arrx[i][0] > arrx[i + 1][0]:
                arrx[i], arrx[i + 1] = arrx[i + 1], arrx[i]
                parents[i],parents[i+1] = parents[i+1],parents[i]
                swapped = True
    final = []
    for i in range(len(parents)):
        final.append([np.sum(cost*parents[i]),parents[i]])
    return final

- Initialize Parent Generation

In [ ]:
def init_Parent_Gen(population,num_supply,num_demand,source,dest,cost):
    pi = []
    for i in range(1,num_supply*num_demand+1):
        pi.append(i)  
    parents= []
    for p in range(population):
        X = np.zeros((num_supply,num_demand))
        s = source.copy()
        d = dest.copy()
        test = pi.copy()
        while(len(test)!=0):
            k = random.choice(test)
            i = int(((k-1)/len(d)))
            j = ((k-1)%len(d)) 
            X[i,j] = min(s[i],d[j])
            s[i] = s[i] - X[i,j]
            d[j] = d[j] - X[i,j]
            test.remove(k)
        parents.append([np.sum(cost*X),X])   
    return parents

- Remove copies in the generation

In [ ]:
def remove_copies(parents):
    x = []
    final = []
    for i in range(len(parents)):
        x.append(parents[i][0])
        if i==0:
            final.append(parents[i])
        if i!=0 and x[i]!=x[i-1]:
            final.append(parents[i])
    return final

- Roulette's selection

In [ ]:
def selection(parents):
    F = 0
    for i in range(len(parents)):
        F = F+ parents[i][0]
    prb = []
    for i in range(len(parents)):
        prb.append((F- parents[i][0])/(F*(len(parents)-1)))
    cum_prb = []
    for i in range(len(prb)):
        if i==0:
            cum_prb.append(prb[i])
        else:
            cum_prb.append(cum_prb[-1]+prb[i])
    r = random.random()
    error = 1
    e_ind = 1
    for i in range(len(cum_prb)):
        if error>abs(cum_prb[i]-r):
            error=abs(cum_prb[i]-r)
            e_ind = i
    return parents[e_ind][1]

- Crossover

In [ ]:
def crossover(p,q,num_supply,num_demand,cost):
    D = np.zeros((num_supply,num_demand))
    R = np.zeros((num_supply,num_demand))
    for i in range(num_supply):
        for j in range(num_demand):
            D[i,j] = int((p[i,j]+q[i,j])/2)
            R[i,j] = (p[i,j]+q[i,j])%2
    horP_sum = np.sum(p,axis=1)
    verP_sum = np.sum(q,axis=0)
    R_dash = []
    R_costs = []
    for i in range(100):
        A = np.zeros((num_supply,num_demand))
        test = []
        s = np.sum(R/2,axis=1)
        d = np.sum(R/2,axis=0)
        for i in range(1,len(s)*len(d)+1):
            test.append(i)
        while(len(test)!=0):
            k = random.choice(test)
            i = int(((k-1)/len(d)))
            j = ((k-1)%len(d))
            A[i,j]=0
            if s[i]!=0 and d[j]!=0:
                    A[i,j] = 1
                    s[i] = s[i]-1
                    d[j]= d[j]-1
            if np.sum(D+A,axis=1)[i]>horP_sum[i]:
                if np.sum(D+A,axis=0)[j]>verP_sum[j]:
                    A[i,j] = 0
            test.remove(k)
        if np.sum(cost*A) not in R_costs:
            R_costs.append(np.sum(cost*A))
            R_dash.append(A)
        if len(R_dash)==2:
            break
    flag = 0
    X1 = D+R_dash[0]
    if len(R_dash)>1:
        X2 = D+R_dash[1]
        flag = 1
        return X1,X2,flag
    else:
        return X1,X1,flag

- Mutation

In [ ]:
def mutate(a):
    n_rows, n_cols = random.randint(2,a.shape[0]),random.randint(2,a.shape[1])
    row = []
    col = []
    while len(row)<n_rows:
        x = random.randint(0,a.shape[0]-1)
        if x not in row:
            row.append(x)
    while len(col)<n_cols:
        x = random.randint(0,a.shape[1]-1)
        if x not in col:
            col.append(x)
    row.sort()
    col.sort()
    A = np.zeros((n_rows,n_cols))
    s = np.sum(a[np.ix_(row,col)],axis=1)
    d = np.sum(a[np.ix_(row,col)],axis=0)   
    test = []
    for i in range(1,len(s)*len(d)+1):
        test.append(i)    
    while(len(test)!=0):
        k = random.choice(test)
        i = int(((k-1)/len(d)))
        j = ((k-1)%len(d)) 
        A[i,j] = min(s[i],d[j])
        s[i] = s[i] - A[i,j]
        d[j] = d[j] - A[i,j]
        test.remove(k) 
    row_itr = 0
    col_itr = 0
    for i in row:
        for j in col:
            a[i,j] = A[row_itr,col_itr]            
            col_itr = col_itr+1
        row_itr = row_itr+1
        col_itr=0
    return a

- Give offsprings from nth generation

In [ ]:
def do_cross(copy_parents,cross_num,num_supply,num_demand,cost,source,dest):
    offsprings = []
    parents = add_or_remove(copy_parents,1,cost)
    n = int(cross_num*len(copy_parents))
    while n!=1 and n!=0:
        a = selection(parents)
        b = selection(parents)
        if np.sum(cost*a)!=np.sum(cost*b):
            off1 = np.zeros((num_supply,num_demand))
            off2 = np.zeros((num_supply,num_demand))
            off1,off2,flag = crossover(a,b,num_supply,num_demand,cost)
            temp_hor = 0
            temp_vert = 0
            if flag==1:
                horOff1 = np.sum(off1,axis=1)
                vertOff1 = np.sum(off1,axis=0)
                horOff2 = np.sum(off2,axis=1)
                vertOff2 = np.sum(off2,axis=0)
                for i in range(num_supply):
                    if source[i]==horOff1[i] and source[i]==horOff2[i]:
                        temp_hor = temp_hor+1
                for i in range(num_demand):
                    if dest[i]==vertOff1[i] and dest[i]==vertOff2[i]:
                        temp_vert = temp_vert+1
                if temp_hor==num_supply and temp_vert==num_demand:
                    offsprings.append(off1)
                    offsprings.append(off2)
                n=n-2
            else:
                horOff1 = np.sum(off1,axis=1)
                vertOff1 = np.sum(off1,axis=0)
                for i in range(num_supply):
                    if source[i]==horOff1[i]:
                        temp_hor = temp_hor+1
                for i in range(num_demand):
                    if dest[i]==vertOff1[i]:
                        temp_vert = temp_vert+1
                if temp_hor==num_supply and temp_vert==num_demand:
                    offsprings.append(off1)
                n=n-1
    return offsprings

- Add/Remove cost

In [ ]:
def add_or_remove(parents,flag,cost):
    p = []
    if flag==1:
        for i in range(len(parents)):
            p.append([np.sum(cost*parents[i]),parents[i]])
    if flag==0:
        for i in range(len(parents)):
            p.append(parents[i][1])
    return p

In [ ]:
def do_mutate(copy_parents,mut_num,cost,num_supply,num_demand,source,dest):
    n = int(mut_num*len(copy_parents))
    parents = add_or_remove(copy_parents,1,cost)
    offsprings = []
    temp_hor = 0
    temp_vert = 0
    while n!=0:
        a = selection(parents)
        off1 = mutate(a)
        horOff1 = np.sum(off1,axis=1)
        vertOff1 = np.sum(off1,axis=0)
        for i in range(num_supply):
            if source[i]==horOff1[i]:
                temp_hor = temp_hor+1
        for i in range(num_demand):
            if dest[i]==vertOff1[i]:
                temp_vert = temp_vert+1
        if temp_hor==num_supply and temp_vert==num_demand:
            offsprings.append(off1)
        n = n-1
    return offsprings

#### Wrapping all the required functions and steps in a single function

In [ ]:
def GA(supply_coords,demand_coords,source,dest,population=100,cross_num=0.4,mutate_num=0.2,num_gen=1000,convergence_no=25):
  num_supply = len(source)
  num_demand = len(dest)

  start = time.time()
  cost = find_cost(supply_coords,demand_coords)
  parents = init_Parent_Gen(population,num_supply,num_demand,source,dest,cost)
  parents = bubble_sort(parents,cost)
  parents= remove_copies(parents)
  no = len(parents)
  p = parents.copy()
  itr = 0
  temp = []
  endFinder = 200
  endNum =0
  c=[]
  m=[]


  for i in range(num_gen):

    temp = []
    c = add_or_remove(do_cross(add_or_remove(p,0,cost),cross_num,num_supply,num_demand,cost,source,dest),1,cost)
    m = add_or_remove(do_mutate(add_or_remove(p,0,cost),mutate_num,cost,num_supply,num_demand,source,dest),1,cost)
    
    temp = c+m+p

    temp = bubble_sort(temp,cost)
    temp = remove_copies(temp)
    p = []
    p = temp[:population]

    itr = itr+1
    if endFinder>p[0][0]:
        endFinder=p[0][0]
        if endNum==0:
            endNum= endNum+1
        else:
            endNum=0
    elif endFinder==p[0][0]:
        endNum= endNum+1
        if (endNum-1)==convergence_no:
            break
  end = time.time()

  print('Optimized cost:',p[0][0],'\n','Solution matrix:','\n',p[0][1],'\n','Time taken:',end-start,'sec')
  opt_cost, sol_mat, time_taken = p[0][0], p[0][1], end-start
  return opt_cost, sol_mat, time_taken

### Benchmarking using PuLP

In [ ]:
% pip install pulp

# Import PuLP modeler functions
from pulp import *

In [ ]:
def concat(l1,l2):
  ''' function to concatenate 2 list of lists, l1 and l2 here, to a single list of lists'''
  temp = l1.copy()
  for i in range(len(l2)):
    temp.append(l2[i]) 
  return temp

def cost_array(coords, prop_const=1, norm='euclidean',lower_cost=1,upper_cost=10):
  ''' function to calculate a flattened cost matrix from the coordinates of the nodes and the type of norm specified'''
  a = []
  if norm == 'euclidean':
    for i in range(len(coords)):
      for j in range(len(coords)):
        if i != j:
          a.append(prop_const*np.linalg.norm(np.asarray(coords[i])-np.asarray(coords[j])))
    return a
  elif norm == 'random':
    return [lower_cost + (upper_cost-lower_cost)*random.random() for i in range(len(coords)*(len(coords)-1))]
  else:
    print('CHOOSE SUITABLE NORM FOR COST')

In [ ]:
def solve_pulp(supply_coords,demand_coords,source,dest,prop_const=1, norm='euclidean',flow_bound_lower=0,flow_bound_upper=100,lower_cost=1,upper_cost=10):
  '''Function to solve the LP problem using PuLP'''

  coords = concat(supply_coords, demand_coords) # concatenating supply and demand coordinates to a single list of lists
  n = len(coords)
  COSTS = cost_array(coords) # flattened cost matrix
  C = COSTS.copy()

  start = time.time()
  EDGES = []
  for i in range(n*(n-1)):
    EDGES.append((flow_bound_lower,flow_bound_upper)) # setting uniform lower/upper flow bound for all xij
  B = source + list(map(lambda x: -x, dest)) # concatenated list of supply and demand values with demand values signed negative
  

  # Model
  prob = LpProblem("Problem",LpMinimize)

  # var_vect is variable matrix(excluding diagonal elements) flattened to a list of 'LpVariable' class objects
  var_vect = []
  edge_count = 0
  for row in range(n):
    for col in range(n):
      if row != col:
        var_vect.append(LpVariable('x'+str(row)+'_'+str(col), *EDGES[edge_count]))
        edge_count += 1

  # Objective function
  prob += lpDot(COSTS, var_vect), "Costs"
  
  # n constraints: Sum of inflow and outflow through a node should be equal to the supply or demand of that node
  for i in range(n):
    for j in range(n):
      if i != j:
        if i<j:
          if (i==0 and j==1) or (i!=0 and j==0):
            lhs = var_vect[(n-1)*i + j-1] - var_vect[(n-1)*j + i]
          else:
            lhs += var_vect[(n-1)*i + j-1] - var_vect[(n-1)*j + i]
        else:
          if (i==0 and j==1) or (i!=0 and j==0):
            lhs = var_vect[(n-1)*i + j] - var_vect[(n-1)*j + i-1]
          else:
            lhs += var_vect[(n-1)*i + j] - var_vect[(n-1)*j + i-1]
    prob += lhs == B[i]

  # Solve
  status = prob.solve()
  
  # Print solution status
  print(LpStatus[status])
  end = time.time()
  
  opt_cost = value(prob.objective)
  print('Optimized cost:', opt_cost)

  # Storing the optimized values of variables in a list
  vars = []
  for var in var_vect:
    vars.append(value(var))
 
  # retrieving parameter matrix(Cij) and variable matrix(xij) from their flattened versions(C and vars respectively) 
  Cij = np.zeros([n,n])
  xij = np.zeros([n,n])
  count = 0
  for row in range(n):
    for col in range(n):
      if row != col:
        Cij[row][col] = C[count]
        xij[row][col] = vars[count]
        count += 1

  # keeping only the edges between supply and demand nodes
  Cij = np.asarray(Cij)[:len(supply_coords), -len(demand_coords):].tolist()
  xij = np.asarray(xij)[:len(supply_coords), -len(demand_coords):].tolist()

  print('Parameter matrix:','\n', np.asarray(Cij))
  print('\n')
  print('Variable matrix:','\n', np.asarray(xij))

  print("Time taken = ", end-start)

  return opt_cost, Cij, xij, end-start

#### INPUT 

In [ ]:
# INPUT 0
source = [8,4,12,6]
dest = [3,5,10,7,5]
supply_coords = [[7., 6.],[2., 7.],[1., 6.],[6., 8.]]
demand_coords = [[3., 9.],[4., 6.],[5., 7.],[5., 3.],[2., 9.]]

# INPUT 1
source1 = [13,12,10,5,3,8,1,2,17]
dest1 = [9, 16,8,9,2,5,18,2,2]
supply_coords1 = []
demand_coords1 = []
#randomly initializing the coordinates
for node_s in range(len(source1)):
  supply_coords1.append([50*random.random(),50*random.random()])
for node_d in range(len(dest1)):
  demand_coords1.append([50*random.random(),50*random.random()])

# INPUT 2
source2 = list(range(1,25))
dest2 = list(range(1,25))
supply_coords2 = []
demand_coords2 = []
#randomly initializing the coordinates
for node_s in range(len(source2)):
  supply_coords2.append([50*random.random(),50*random.random()])
for node_d in range(len(dest2)):
  demand_coords2.append([50*random.random(),50*random.random()])


 ### Solutions

*   #### Solutions via genetic algorithm




In [ ]:
print('Solution to input 0:','\n\n')
opt_cost_ga, sol_matrix_ga, time_taken_ga = GA(supply_coords,demand_coords,source,dest)
print('---------------------------')
print('Solution to input 1:','\n\n')
opt_cost_ga_1, sol_matrix_ga_1, time_taken_ga_1 = GA(supply_coords1,demand_coords1,source1,dest1)
print('---------------------------')
print('Solution to input 2:','\n\n')
opt_cost_ga_2, sol_matrix_ga_2, time_taken_ga_2 = GA(supply_coords2,demand_coords2,source2,dest2)
print('---------------------------')


Solution to input 0: 


Optimized cost: 83.20907295926658 
 Solution matrix: 
 [[0. 0. 4. 4. 0.]
 [3. 0. 0. 0. 1.]
 [0. 5. 0. 3. 4.]
 [0. 0. 6. 0. 0.]] 
 Time taken: 8.504777193069458 sec
---------------------------
Solution to input 1: 


Optimized cost: 670.5537978024969 
 Solution matrix: 
 [[ 0.  0.  0.  0.  0.  0. 13.  0.  0.]
 [ 0. 12.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  5.  0.  0.  5.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  2.  2.]
 [ 0.  0.  2.  0.  1.  0.  0.  0.  0.]
 [ 0.  4.  0.  0.  0.  0.  4.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  1.  1.  0.  0.  0.  0.  0.]
 [ 9.  0.  0.  8.  0.  0.  0.  0.  0.]] 
 Time taken: 59.4437689781189 sec
---------------------------
Solution to input 2: 


Optimized cost: 6254.20919718621 
 Solution matrix: 
 [[ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  2.]
 [ 0.  0. 

*   #### Solutions via PuLP library




In [ ]:
print('Solution to input 0:','\n\n')
opt_cost_pulp, param_matrix, sol_matrix_pulp, time_taken_pulp = solve_pulp(supply_coords,demand_coords,source,dest)
print('---------------------------')
print('Solution to input 1:','\n\n')
opt_cost_pulp_1, param_matrix_1, sol_matrix_pulp_1, time_taken_pulp_1 = solve_pulp(supply_coords1,demand_coords1,source1,dest1)
print('---------------------------')
print('Solution to input 2:','\n\n')
opt_cost_pulp_2, param_matrix_2, sol_matrix_pulp_2, time_taken_pulp_2 = solve_pulp(supply_coords2,demand_coords2,source2,dest2)
print('---------------------------')

Solution to input 0: 


Optimal
Optimized cost: 83.20907295926656
Parameter matrix: 
 [[5.         3.         2.23606798 3.60555128 5.83095189]
 [2.23606798 2.23606798 3.         5.         2.        ]
 [3.60555128 3.         4.12310563 5.         3.16227766]
 [3.16227766 2.82842712 1.41421356 5.09901951 4.12310563]]


Variable matrix: 
 [[0. 0. 4. 4. 0.]
 [3. 0. 0. 0. 1.]
 [0. 5. 0. 3. 4.]
 [0. 0. 6. 0. 0.]]
Time taken =  0.03031158447265625
---------------------------
Solution to input 1: 


Optimal
Optimized cost: 1213.023464937906
Parameter matrix: 
 [[22.93767956 30.14662019 27.58369321 17.89723346 29.64384406 23.08258243
  19.29984079 16.98374821 22.94007308]
 [33.74304886 44.78467028 53.09414905 41.82784626 31.06722856  6.76278165
  36.49825157 46.31614696 15.37025972]
 [23.87358259 33.6446066  36.14395538 24.50244563 27.26965838 12.80162079
  21.97747522 27.27548341 15.49202773]
 [30.20393577 37.87453224 34.30354671 13.34879278 35.99914201 23.16848711
  12.04376196 21.62925506 

Functions required for the Simplex Big-M Algo

In [ ]:
import time
#Calculating the Relative Profits of all the columns
def zj_cj(l, cj):
    a = [0, 0]
    for i in range(2, len(cj)):
        a.append(-cj[i])
        
    for i in range(len(l)):
        for j in range(2, len(cj)):
            a[j] += l[i][j]*l[i][0]
    return a.index(min(a)), min(a)
#Calculating the ratio of the selected columns and finding the minimum
def min_ratio(l, cj):
    m = 10000000000
    a = list()
    b = zj_cj(l, cj)[0]
    for i in range(len(l)):
        if l[i][b] > 0:
            a.append([l[i][1]/l[i][b], l[i][0]])
        else:
            a.append([m, l[i][0]])
    x = a[0][0]
    y = a[0][1]
    z = 0
    for i in range(1, len(l)):
        if x > a[i][0]:
            x = a[i][0]
            y = a[i][1]
            z = i
        elif x == a[i][0]:
            if y > a[i][1]:
                y = a[i][1]
                z = i
    return z
#Used Simplex Big-M method to optimize the matrix and finding the optimised cost
#using summation of CbXb
def optimize(l, cj):
    start_time = time.time()
    #x = []
    #n = []
    n_ = 0
    while(zj_cj(l, cj)[1] < 0):
        n_ += 1
        x_ = 0
        a = zj_cj(l, cj)[0]
        b = min_ratio(l, cj)
        for i in range(len(l[0])):
            l[b][i] = l[b][i]/l[b][a]
        l[b][0] = cj[a]
        for i in range(len(l)):
            if (l[i][a] != 0 and i != b):
                y = l[i][a]
                for j in range(1, len(l[0])):
                    l[i][j] = l[i][j] - (y*l[b][j])
        for i in range(len(l)):
            x_ += l[i][0]*l[i][1]
        #x.append(x_)
        #n.append(n_)
    #plt.plot(n, x)
    #plt.scatter(n, x)
    #plt.xlabel("No. of iterations")
    #plt.ylabel("Optimized Cost")
    #plt.title("1521 variables")
    x_ = str(x_)
    print("--- %s seconds ---" % (time.time() - start_time))
    return "Optimized Cost = " + x_, l

#print(optimize(l, cj))

All Functions of Simplex Big-M combined

In [ ]:
def Simplex(S, D, dist):
    l = list()
    for i in range(len(S) + len(D)):
        l.append([])

    M = 10000000000
    c = 1

    cj = [0, 0]
    for i in range(len(S)):
        for j in range(len(D)):
            cj.append(c*dist[i][j])
    for i in range(len(S) + len(D)):
        cj.append(0)
    for i in range(len(D)):
        cj.append(-M)
        
    for i in range(len(S) + len(D)):
        if (i < len(S)):
            l[i].append(0)
            l[i].append(S[i])
            for j in range(len(S)*len(D)):
                if (i == j//len(D)):
                    l[i].append(1)
                else:
                    l[i].append(0)
                    
            for j in range(len(S) + len(D)):
                if (i == j):
                    l[i].append(1)
                else:
                    l[i].append(0)
            
            for j in range(len(D)):
                l[i].append(0)
                
        else:
            l[i].append(-M)
            l[i].append(D[i - len(S)])
            
            for j in range(len(S)):
                for k in range(len(D)):
                    if (k == (i - len(S))):
                        l[i].append(1)
                    else:
                        l[i].append(0)
                        
            for j in range(len(S) + len(D)):
                if (i == j):
                    l[i].append(-1)
                else:
                    l[i].append(0)
            
            for j in range(len(D)):
                if ((i - len(S)) == j):
                    l[i].append(1)
                else:
                    l[i].append(0)
    return optimize(l, cj)

In [ ]:
# INPUT 0
S = [8, 4, 12, 6]
D = [3, 5, 10, 7, 5]
dist = [[5, 3, 2.236068, 3.60555, 5.830952], [2.236068, 2.236068, 3, 5, 2], [3.60555, 3, 4.1231, 5, 3.162278], [3.162278, 2.82843, 1.41421, 5.09902, 3.60555]]

# INPUT 1
source1 = [13,12,10,5,3,8,1,2,17]
dest1 = [9, 16,8,9,2,5,18,2,2]
dist1 = []
#randomly initializing the DISTANCE
for i in range(len(source1)):
    dist1.append([])
    for j in range(len(dest1)):
        dist1[i].append(20*random.random())

# INPUT 2
source2 = list(range(1,25))
dest2 = list(range(1,25))
dist2 = []
#randomly initializing the DISTANCE
for i in range(len(source2)):
    dist2.append([])
    for j in range(len(dest2)):
        dist2[i].append(50*random.random())

In [ ]:
print("Solution to Input 0")
print(Simplex(S, D, dist))
print("Solution to Input 1")
print(Simplex(source1, dest1, dist1))
print("Solution to Input 2")
print(Simplex(source2, dest2, dist2))

Solution to Input 0
--- 0.0046405792236328125 seconds ---
('Optimized Cost = 135.16811', [[3, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, -1.0, 0.0, 0.0, 0.0, -1.0, -1.0, 0.0, 0.0, 0.0, -1.0, -1.0, 0.0, 0.0, 0.0, -1.0, 0.0, -1.0, -1.0, -1.0, 0.0, -1.0, -1.0, -1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0], [0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, -1.0], [3, 2.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0], [5.09902, 3.0, 0.0, 0.0, 0.0, 1.0, 0.0, -1.0, -1.0, -1.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0], [5, 3.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 0.0, 1.

Comparison of Simplex Big-M, Genetic and Pulp giving the same non-integral input

In [ ]:
print('Solution to input 0 via Pulp:','\n\n')
opt_cost_pulp, param_matrix, sol_matrix_pulp, time_taken_pulp = solve_pulp(supply_coords,demand_coords,source,dest)
print("Solution to Input 0 via Simplex Big-M")
print(Simplex(S, D, dist))
print('Solution to input 0 via GA:','\n\n')
opt_cost_ga, sol_matrix_ga, time_taken_ga = GA(supply_coords,demand_coords,source,dest)

Solution to input 0 via Pulp: 


Optimal
Optimized cost: 83.20907295926656
Parameter matrix: 
 [[5.         3.         2.23606798 3.60555128 5.83095189]
 [2.23606798 2.23606798 3.         5.         2.        ]
 [3.60555128 3.         4.12310563 5.         3.16227766]
 [3.16227766 2.82842712 1.41421356 5.09901951 4.12310563]]


Variable matrix: 
 [[0. 0. 4. 4. 0.]
 [3. 0. 0. 0. 1.]
 [0. 5. 0. 3. 4.]
 [0. 0. 6. 0. 0.]]
Time taken =  0.030795812606811523
Solution to Input 0 via Simplex Big-M
--- 0.002275705337524414 seconds ---
('Optimized Cost = 135.16811', [[3, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, -1.0, 0.0, 0.0, 0.0, -1.0, -1.0, 0.0, 0.0, 0.0, -1.0, -1.0, 0.0, 0.0, 0.0, -1.0, 0.0, -1.0, -1.0, -1.0, 0.0, -1.0, -1.0, -1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0], [0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, -1.0], [3, 2.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0

Test Example 1

In [ ]:
S1_ = [3, 6]
D1_ = [2, 4]
dis1_ = [[1, 3], [2, 5]]
sup_coords = [[7., 6.], [6., 8.]]
dema_coords = [[7, 5], [8.538, 3.513]]
opt_cost_pulp, param_matrix, sol_matrix_pulp, time_taken_pulp = solve_pulp(sup_coords, dema_coords, S1_, D1_)
print(Simplex(S1_, D1_, dis1_))

Infeasible
Optimized cost: 35.71721281416264
Parameter matrix: 
 [[1.         2.92414312]
 [3.16227766 5.15505703]]


Variable matrix: 
 [[0. 3.]
 [2. 4.]]
Time taken =  0.026416301727294922
--- 0.0002918243408203125 seconds ---
('Optimized Cost = 35.0', [[3, 1.0, 0.0, 1.0, -1.0, 0.0, 1.0, 0.0, 1.0, 0.0, -1.0, 0.0], [0, 3.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, -1.0, -1.0], [1, 2.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 1.0, 0.0], [5, 6.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]])


Test Example 2

In [ ]:
S1 = [8,4,12,6]
D1 = [3,5,10,7,5]
dist1_ = [[1, 2, 3, 4, 5], [2, 3, 4, 5, 6], [3, 4, 5, 6, 7], [4, 5, 6, 7, 8]]
supp_coords = [[0, 0], [0, -1], [0, -2], [0, -3]]
dem_coords = [[0, 1], [0, 2], [0, 3], [0, 4], [0, 5]]
print('Solution to input 0 via Pulp:','\n\n')
opt_cost_pulp, param_matrix, sol_matrix_pulp, time_taken_pulp = solve_pulp(supp_coords, dem_coords, S1, D1)
print(Simplex(S1, D1, dist1_))

Solution to input 0 via Pulp: 


Optimal
Optimized cost: 142.0
Parameter matrix: 
 [[1. 2. 3. 4. 5.]
 [2. 3. 4. 5. 6.]
 [3. 4. 5. 6. 7.]
 [4. 5. 6. 7. 8.]]


Variable matrix: 
 [[0. 4. 0. 4. 0.]
 [0. 0. 4. 0. 0.]
 [0. 1. 6. 0. 5.]
 [3. 0. 0. 3. 0.]]
Time taken =  0.02647876739501953
--- 0.0023725032806396484 seconds ---
('Optimized Cost = 142.0', [[0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, -1.0], [3, 0.0, 0.0, 0.0, -1.0, -1.0, -1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, -1.0, -1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0], [5, 6.0, 0.0, 0.0, 0.0, -1.0, -1.0, 0.0, 0.0, 0.0, -1.0, -1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, -1.0, -1.0, 0.0, 0.0, -1.0, -1.0, -1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0], [7, 1.0, 0.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 0.0, -1.0, 1.0